## Setup

In [1]:
# import modules
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# matplotlib dependencies
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
# sqlalchemy dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

## Reflect Tables into SQLAlchemy ORM

In [4]:
# create engine to database
engine  = create_engine("sqlite:///../Resources/hawaii.sqlite")

# declare a base
Base = automap_base()

# use base class to reflect the database tables
Base.prepare(autoload_with=engine)

In [5]:
# view the classes mapped to the base
Base.classes.keys()

['measurement', 'station']

In [6]:
# assign each class to their own variable
Measurement = Base.classes.measurement
Station = Base.classes.station

In [7]:
# create a session from python to database
session = Session(engine)